In [2]:
import os
import shutil
import json
import polus.plugins
from polus.plugins import plugins
from pathlib import Path
from pprint import pprint

In [3]:
manifests = [
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/feat/basic/regression/polus-basic-flatfield-correction-plugin/plugin.json',
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/feat/applyflatfield/transforms/images/polus-apply-flatfield-plugin/plugin.json',
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/plugin/smp-training/segmentation/polus-smp-training-plugin/plugin.json',
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/ftl/feat/binarization-threshold/transforms/images/polus-ftl-label-plugin/plugin.json',
    'https://raw.githubusercontent.com/hamshkhawar/polus-plugins/file-renaming/formats/polus-file-renaming-plugin/plugin.json',
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/scaled-nyxus/features/scaled-nyxus/plugin.json'    
]

In [4]:
for url in manifests:
    polus.plugins.submit_plugin(url, refresh=True)
    
print(plugins.list)

['ApplyFlatfield', 'BasicFlatfieldCorrectionPlugin', 'DemoSmpTraining_Inference', 'FileRenaming', 'FtlLabel', 'ScaledNyxus']


In [5]:
int_dir = Path('C:/Users/benho/polus-plugins/data/eastman/intensity')
assert int_dir.exists(), "The input directory does not exist"

int_files = [file.name for file in int_dir.iterdir()]
int_files.sort()
print('The number of input images is {}'.format(len(int_files)))
print('\nThe first 5 images:')
pprint(int_files[:5])
print('\nThe last 5 images:')
pprint(int_files[-5:])

file_pattern = 'x0{x}_y0{y}_wx{t}_wy{p}_c{c}.ome.tif'

The number of input images is 36

The first 5 images:
['x01_y01_wx0_wy0_c1.ome.tif',
 'x01_y01_wx0_wy0_c2.ome.tif',
 'x01_y01_wx0_wy1_c1.ome.tif',
 'x01_y01_wx0_wy1_c2.ome.tif',
 'x01_y01_wx0_wy2_c1.ome.tif']

The last 5 images:
['x01_y02_wx2_wy0_c2.ome.tif',
 'x01_y02_wx2_wy1_c1.ome.tif',
 'x01_y02_wx2_wy1_c2.ome.tif',
 'x01_y02_wx2_wy2_c1.ome.tif',
 'x01_y02_wx2_wy2_c2.ome.tif']


In [6]:
out_dir = {}
parent_out_dir = Path('C:/Users/benho/polus-plugins/data/eastman')
workflow = [
    'BasicFlatfieldCorrectionPlugin',
    'ApplyFlatfield',
    'DemoSmpTraining_Inference',
    'FtlLabel',
    'FileRenaming',
    'ScaledNyxus'
    ]

if not parent_out_dir.exists():
    os.mkdir(parent_out_dir)

for name in workflow:
    out_dir[name] = parent_out_dir.joinpath(name)
    if out_dir[name].exists():
        continue
        ### CAUTION THIS REMOVES OLD OUT DIRECTORIES
        shutil.rmtree(out_dir[name])
    elif name :
        os.mkdir(out_dir[name])
    
pprint(out_dir)

{'ApplyFlatfield': WindowsPath('C:/Users/benho/polus-plugins/data/eastman/ApplyFlatfield'),
 'BasicFlatfieldCorrectionPlugin': WindowsPath('C:/Users/benho/polus-plugins/data/eastman/BasicFlatfieldCorrectionPlugin'),
 'DemoSmpTraining_Inference': WindowsPath('C:/Users/benho/polus-plugins/data/eastman/DemoSmpTraining_Inference'),
 'FileRenaming': WindowsPath('C:/Users/benho/polus-plugins/data/eastman/FileRenaming'),
 'FtlLabel': WindowsPath('C:/Users/benho/polus-plugins/data/eastman/FtlLabel'),
 'ScaledNyxus': WindowsPath('C:/Users/benho/polus-plugins/data/eastman/ScaledNyxus')}


In [7]:
flatfield_corr = plugins.BasicFlatfieldCorrectionPlugin
apply_flatfield = plugins.ApplyFlatfield
ftl = plugins.FtlLabel
smp = plugins.DemoSmpTraining_Inference
file_renaming = plugins.FileRenaming
nyxus = plugins.ScaledNyxus

In [8]:
[inp.name for inp in flatfield_corr.inputs + flatfield_corr.outputs]

['inpDir', 'filePattern', 'darkfield', 'photobleach', 'groupBy', 'outDir']

In [9]:
flatfield_corr.inpDir = int_dir
flatfield_corr.filePattern = file_pattern
flatfield_corr.darkfield = True
flatfield_corr.photobleach = False
flatfield_corr.groupBy = 'xytp'
flatfield_corr.outDir = str(out_dir['BasicFlatfieldCorrectionPlugin'])

for inp in flatfield_corr.inputs + flatfield_corr.outputs:
    print('{} = {}'.format(inp.name, inp.value))

inpDir = C:\Users\benho\polus-plugins\data\eastman\intensity
filePattern = x0{x}_y0{y}_wx{t}_wy{p}_c{c}.ome.tif
darkfield = True
photobleach = False
groupBy = xytp
outDir = C:\Users\benho\polus-plugins\data\eastman\BasicFlatfieldCorrectionPlugin


In [10]:
flatfield_corr.run()

17-Oct-22 12:20:14 - polus.plugins - INFO     - Running container with GPU: --gpus all. BasicFlatfieldCorrectionPlugin version 1.3.1


In [11]:
[inp.name for inp in apply_flatfield.inputs + apply_flatfield.outputs]

['darkPattern',
 'ffDir',
 'brightPattern',
 'imgDir',
 'imgPattern',
 'photoPattern',
 'outDir']

In [12]:
apply_flatfield.imgDir = int_dir
apply_flatfield.imgPattern = file_pattern
apply_flatfield.ffDir = out_dir['BasicFlatfieldCorrectionPlugin']
apply_flatfield.brightPattern = 'x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_flatfield.ome.tif'
apply_flatfield.darkPattern = 'x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_darkfield.ome.tif'
apply_flatfield.outDir = out_dir['ApplyFlatfield']

for inp in apply_flatfield.inputs + apply_flatfield.outputs:
    print('{} = {}'.format(inp.name, inp.value))

darkPattern = x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_darkfield.ome.tif
ffDir = C:\Users\benho\polus-plugins\data\eastman\BasicFlatfieldCorrectionPlugin
brightPattern = x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_flatfield.ome.tif
imgDir = C:\Users\benho\polus-plugins\data\eastman\intensity
imgPattern = x0{x}_y0{y}_wx{t}_wy{p}_c{c}.ome.tif
photoPattern = None
outDir = C:\Users\benho\polus-plugins\data\eastman\ApplyFlatfield


In [13]:
apply_flatfield.run()

17-Oct-22 12:20:27 - polus.plugins - INFO     - Running container with GPU: --gpus all. ApplyFlatfield version 1.2.0


In [14]:
[inp.name for inp in smp.inputs + smp.outputs]

['inferenceMode',
 'imagesInferenceDir',
 'inferencePattern',
 'pretrainedModel',
 'modelName',
 'encoderBase',
 'encoderVariant',
 'encoderWeights',
 'optimizerName',
 'batchSize',
 'imagesTrainDir',
 'labelsTrainDir',
 'trainPattern',
 'imagesValidDir',
 'labelsValidDir',
 'validPattern',
 'device',
 'checkpointFrequency',
 'lossName',
 'maxEpochs',
 'patience',
 'minDelta',
 'outputDir']

In [15]:
smp_file_pattern = 'x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif'

smp.imagesInferenceDir = out_dir['ApplyFlatfield']
smp.inferenceMode = 'active'
smp.inferencePattern = smp_file_pattern
smp.pretrainedModel = Path('C:/Users/benho/polus-plugins/data/smp-models/UnetPlusPlus-MCCLoss-efficientnet-b6-imagenet-Adam/')
smp.batchSize = 10
smp.device = 'gpu'
smp.lossName = 'MCCLoss'
smp.checkpointFrequency = 10
smp.maxEpochs = 10
smp.patience = 10
smp.minDelta = 10.0
smp.outputDir = out_dir['DemoSmpTraining_Inference']

for inp in smp.inputs + smp.outputs:
    print('{} = {}'.format(inp.name, inp.value))

inferenceMode = inferenceMode.active
imagesInferenceDir = C:\Users\benho\polus-plugins\data\eastman\ApplyFlatfield
inferencePattern = x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif
pretrainedModel = C:\Users\benho\polus-plugins\data\smp-models\UnetPlusPlus-MCCLoss-efficientnet-b6-imagenet-Adam
modelName = None
encoderBase = None
encoderVariant = None
encoderWeights = None
optimizerName = None
batchSize = 10
imagesTrainDir = None
labelsTrainDir = None
trainPattern = None
imagesValidDir = None
labelsValidDir = None
validPattern = None
device = gpu
checkpointFrequency = 10
lossName = lossName.MCCLoss
maxEpochs = 10
patience = 10
minDelta = 10.0
outputDir = C:\Users\benho\polus-plugins\data\eastman\DemoSmpTraining_Inference


In [16]:
smp.run()

17-Oct-22 12:20:31 - polus.plugins - INFO     - Running container with GPU: --gpus all. DemoSmpTraining_Inference version 0.5.9


In [17]:
[inp.name for inp in ftl.inputs + ftl.outputs]

['inpDir', 'connectivity', 'binarizationThreshold', 'outDir']

In [18]:
ftl.inpDir = out_dir['DemoSmpTraining_Inference']
ftl.connectivity = 1
ftl.binarizationThreshold = 0.5
ftl.outDir = out_dir['FtlLabel']

for inp in ftl.inputs + ftl.outputs:
    print('{} = {}'.format(inp.name, inp.value))

inpDir = C:\Users\benho\polus-plugins\data\eastman\DemoSmpTraining_Inference
connectivity = 1
binarizationThreshold = 0.5
outDir = C:\Users\benho\polus-plugins\data\eastman\FtlLabel


In [19]:
ftl.run()

17-Oct-22 12:22:15 - polus.plugins - INFO     - Running container with GPU: --gpus all. FtlLabel version 0.3.11


In [20]:
[inp.name for inp in file_renaming.inputs + file_renaming.outputs]

['filePattern', 'inpDir', 'outFilePattern', 'outDir']

In [21]:
smp_file_pattern = 'x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif'

file_renaming.inpDir = out_dir['FtlLabel']
file_renaming.outDir = out_dir['FileRenaming']
file_renaming.filePattern = 'x0{row:d}_y0{col:d}_wx{well_row:d}_wy{well_col:d}_c1.ome.tif'
file_renaming.outFilePattern = 'x0{row:d}_y0{col:d}_wx{well_row:d}_wy{well_col:d}_c2.ome.tif'

for inp in file_renaming.inputs + file_renaming.outputs:
    print('{} = {}'.format(inp.name, inp.value))

filePattern = x0{row:d}_y0{col:d}_wx{well_row:d}_wy{well_col:d}_c1.ome.tif
inpDir = C:\Users\benho\polus-plugins\data\eastman\FtlLabel
outFilePattern = x0{row:d}_y0{col:d}_wx{well_row:d}_wy{well_col:d}_c2.ome.tif
outDir = C:\Users\benho\polus-plugins\data\eastman\FileRenaming


In [22]:
file_renaming.run()

17-Oct-22 12:22:21 - polus.plugins - INFO     - Running container with GPU: --gpus all. FileRenaming version 0.1.17


In [23]:
[inp.name for inp in nyxus.inputs + nyxus.outputs]

['inpDir',
 'segDir',
 'features',
 'filePattern',
 'neighborDist',
 'pixelPerMicron',
 'outDir']

In [24]:
nyxus.inpDir = out_dir['ApplyFlatfield']
nyxus.segDir = out_dir['FileRenaming']
nyxus.features = 'INTEGRATED_INTENSITY,MEAN,AREA_PIXELS_COUNT'
nyxus.filePattern = 'x{x+}.*c2.ome.tif' # x([0-9]+).*c2.ome.tif
nyxus.neighborDist = 5
nyxus.pixelPerMicron = 0.83612
nyxus.outDir = out_dir['ScaledNyxus']

# 'x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif'

In [25]:
nyxus.run()

17-Oct-22 12:22:24 - polus.plugins - INFO     - Running container with GPU: --gpus all. ScaledNyxus version 0.1.0
